In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [2]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user06/beaver/test_data/db"
    save_path = ""
    embedding_model="BAAI/bge-m3"
    retriever_k=4
    retriever_bert_weight=0.7
    version=8.5
    seed=42
    
# CFG.save_path = CFG.output_path + CFG.store + "_temp.csv"

In [9]:
data_df = pd.read_excel(f'/home/user06/beaver/data/beaver_v{CFG.version}.xlsx').rename(columns = {
    '상점명' : 'store', 
    '이전 슬롯' : 'before_slot', 
    '현재 슬롯' : 'current_slot', 
    '이전 입력' : 'before_input', 
    '현재 입력' : 'current_input',
    '이전 응대' : 'before_response',
    '현재 응대' : 'current_response'
})

data_df['store'] = data_df['store'].apply(lambda x: x.split("_")[0])

NameError: name 'pd' is not defined

In [10]:
store_list = ['프랭크버거','고봉민김밥','포트캔커피','아비꼬','피자먹다']
data_df___ = pd.DataFrame()

for store in tqdm(store_list, total=len(store_list)):
    data_df_ = data_df[data_df['store'] == store]

    df = pd.read_excel("/content/범용화_CES용_상점_상품_정보_231017_prepro.xlsx")
    df = df[df['CATEGORY_NM'] != '모바일 상품권']
    # df['OPTION_GROUP_NM'] = df['OPTION_GROUP_NM'].apply(lambda x: "음료 선택" if x=="음료선택" else x)
    df_ = df[df['STORE_NM'] == f"{store}_음성인식챗봇"]

    df_ = df_[['PROD_NM', 'CATEGORY_NM', 'OPTION_GROUP_NM', 'ESS_YN', 'MAX_SEL_CNT', 'OPTION_PROD_NM']]
    df_ = df_.rename(columns = {
            'PROD_NM' : '상품명',
            'CATEGORY_NM' : '카테고리',
            'OPTION_GROUP_NM' : '옵션그룹',
            'ESS_YN' : '필수여부',
            'MAX_SEL_CNT' : '최대선택수',
            'OPTION_PROD_NM' : '옵션명',
        })
    df_ = df_.groupby(["상품명", "카테고리", "옵션그룹", "필수여부", "최대선택수"])["옵션명"].apply(list).reset_index()

    print('---------------- df_ ----------------')
    print(df_.head(10))
    #           상품명          카테고리      옵션그룹     필수여부  최대선택수                                    옵션명
    # 0        JG 버거         버거 단품     사이즈 선택      Y          1                                         [L, R]
    # 1      JG 버거 세트      버거 세트     사이드 선택      Y          1        [치즈 스틱(3개), 콘샐러드, 피넛 슈가볼, 스파이시 텐더(2개), 더치즈볼(3개...
    # 2      JG 버거 세트      버거 세트     사이즈 선택      Y          1                                         [L, R]
    # 3      JG 버거 세트      버거 세트      음료선택        Y          1        [콜라, 제로 콜라, 사이다, 오렌지, 아메리카노(ICE), 아메리카노(HOT), ...
    # 4    JG 치즈 스틱(1개)    사이드      체다치즈 선택     Y          1                                  [체다 치즈, 크림 치즈]
    # 5     SG 불고기 버거     버거 단품     사이즈 선택      Y          1                                         [L, R]
    # 6  SG 불고기 버거 세트   버거 세트     사이드 선택      Y          1        [치즈 스틱(3개), 콘샐러드, 피넛 슈가볼, 스파이시 텐더(2개), 더치즈볼(3개...
    # 7  SG 불고기 버거 세트   버거 세트     사이즈 선택      Y          1                                         [L, R]
    # 8  SG 불고기 버거 세트   버거 세트      음료선택        Y          1        [콜라, 제로 콜라, 사이다, 오렌지, 아메리카노(ICE), 아메리카노(HOT), ...
    # 9   SG 크림 치즈 버거    버거 단품     사이즈 선택      Y          1                                         [L, R]

    print(df.shape)   # df.shape: (2909, 14)
    print(df_.shape)  # df_.shape: (63, 6)

    menu_str = ""
    for category_name, df_grouped_by_category in df_.groupby("카테고리"):
        print('---------------- category_name ----------------')
        print(category_name)
        # 버거 단품
        # 버거 세트
        # 사이드
        # 음료
        print('---------------- df_grouped_by_category ----------------')
        print(df_grouped_by_category)
        #            상품명         카테고리    옵션그룹     필수여부    최대선택수   옵션명
        # 0         JG 버거        버거 단품   사이즈 선택       Y           1        [L, R]
        # 5     SG 불고기 버거     버거 단품   사이즈 선택       Y           1        [L, R]
        # 9   SG 크림 치즈 버거    버거 단품   사이즈 선택       Y           1        [L, R]
        # 14     더블 치즈 버거    버거 단품   사이즈 선택       Y           1        [L, R]
        # 19       머쉬룸 버거     버거 단품   사이즈 선택       Y           1        [L, R]
        # 24    베이컨 치즈 버거   버거 단품   사이즈 선택       Y           1        [L, R]
        # 30       쉬림프 버거     버거 단품      [NULL]      [NULL]      [NULL]     [[NULL]]
        # 41        치즈 버거      버거 단품   사이즈 선택       Y           1        [L, R]
        # 56       프랭크 버거     버거 단품   사이즈 선택       Y           1        [L, R]

        menu_str += f"- {category_name}: "  # - 버거 단품:
        for prod_nm in df_grouped_by_category['상품명']:
            menu_str += prod_nm + ', '
            print('---------------- menu_str ----------------')
            print(menu_str)
            # - 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 버거, 머쉬룸 버거, 베이컨 치즈 버거, 쉬림프 버거, 치즈 버거, 프랭크 버거,
        menu_str += "\n"

    print('---------------- whole menu_str ----------------')
    print(menu_str)
    # - 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 버거, 머쉬룸 버거, 베이컨 치즈 버거, 쉬림프 버거, 치즈 버거, 프랭크 버거,
    # - 버거 세트: JG 버거 세트, JG 버거 세트, JG 버거 세트, SG 불고기 버거 세트, SG 불고기 버거 세트, SG 불고기 버거 세트, SG 크림 치즈 버거 세트, SG 크림 치즈 버거 세트, SG 크림 치즈 버거 세트, 더블 치즈 버거 세트, 더블 치즈 버거 세트, 더블 치즈 버거 세트, 머쉬룸 버거 세트, 머쉬룸 버거 세트, 머쉬룸 버거 세트, 베이컨 치즈 버거 세트, 베이컨 치즈 버거 세트, 베이컨 치즈 버거 세트, 쉬림프 버거 세트, 쉬림프 버거 세트, 오딘에 반할라 버거 세트, 치즈 버거 세트, 치즈 버거 세트, 치즈 버거 세트, 프랭크 버거 세트, 프랭크 버거 세트, 프랭크 버거 세트,
    # - 사이드: JG 치즈 스틱(1개), 게다리살 튀김(3개), 더치즈볼(3개), 스파이시 텐더(2개), 자이언트 통닭다리(1개), 초코칩 쿠키, 치즈 스틱(3개), 치즈 쿠키, 치즈 프랜치 프라이, 치킨 매운맛(윙2+봉2), 치킨 순한맛(윙2+봉2), 코울슬로 샐러드, 콘 치즈볼(3개), 콘샐러드, 통가슴 후라이드(1개), 프랜치 프라이, 피넛 슈가볼, 후라이드 아이스볼(1개), 후라이드 아이스볼(2개),
    # - 음료: 밀크쉐이크, 사이다, 생수 500ml, 아메리카노(HOT), 아메리카노(ICE), 오렌지, 제로 콜라, 콜라,

    data = {}
    for prod_nm, df_grouped_by_prod_nm in df_.groupby('상품명'):
        sub_data = {}
        for ess_yn, option_group_nm, max_sel_cnt in zip(df_grouped_by_prod_nm['필수여부'], df_grouped_by_prod_nm['옵션그룹'], df_grouped_by_prod_nm['최대선택수']):
            ess_yn = '[필수 주문]' if ess_yn == 'Y' else '[선택 주문]' if ess_yn == 'N' else 'None'
            max_sel_cnt = f'(최대 {max_sel_cnt}개)' if type(max_sel_cnt) == str else ''
            name = f'{ess_yn} {option_group_nm} {max_sel_cnt}'
            sub_data[name] = option_group_nm

        # for j, df___ in df__.iterrows():
        #     group = df___['옵션그룹']
        #     ess = f"[필수 주문]" if df___['필수여부'] == 'Y' else "[선택 주문]" if df___['필수여부'] == 'N' else "None"
        #     num = f"(최대 {df___['최대선택수']}개)" if type(df___['최대선택수'])==str else ""
        #     name = f"{ess} {group} {num}"
        #     sub_data[name] = df___['옵션명']

            if ess_yn == "None":
                data[prod_nm] = "[옵션 없음]"
            else:
                data[prod_nm] = str(sub_data)#.replace("'", "\"")

        print('---------------- sub_data ----------------')
        print(sub_data)
    print('---------------- data ----------------')
    print(data)
    df_2 = pd.DataFrame.from_dict(data, orient='index')#.to_csv(CFG.save_path)
    df_2 = df_2.reset_index()
    df_2.columns = ['상품명', '옵션']

    print('---------------- df_2 ----------------')
    print(df_2)
    #              상품명                                                 옵션
    # 0            JG 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 1         JG 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 2     JG 치즈 스틱(1개)                    {'[필수 주문] 체다치즈 선택 ': '체다치즈 선택'}
    # 3        SG 불고기 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 4     SG 불고기 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 5      SG 크림 치즈 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 6   SG 크림 치즈 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 7      게다리살 튀김(3개)                                            [옵션 없음]
    # 8         더블 치즈 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 9      더블 치즈 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 10        더치즈볼(3개)                                            [옵션 없음]
    # 11          머쉬룸 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 12       머쉬룸 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 13           밀크쉐이크                                            [옵션 없음]
    # 14       베이컨 치즈 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 15    베이컨 치즈 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 16             사이다                                            [옵션 없음]
    # 17        생수 500ml                                            [옵션 없음]
    # 18          쉬림프 버거                                            [옵션 없음]
    # 19       쉬림프 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 음료선택 ':...
    # 20     스파이시 텐더(2개)                                            [옵션 없음]
    # 21      아메리카노(HOT)                                            [옵션 없음]
    # 22      아메리카노(ICE)                                            [옵션 없음]
    # 23   오딘에 반할라 버거 세트                                            [옵션 없음]
    # 24             오렌지                                            [옵션 없음]
    # 25   자이언트 통닭다리(1개)                                            [옵션 없음]
    # 26           제로 콜라                                            [옵션 없음]
    # 27          초코칩 쿠키                                            [옵션 없음]
    # 28           치즈 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 29        치즈 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 30       치즈 스틱(3개)                                            [옵션 없음]
    # 31           치즈 쿠키                                            [옵션 없음]
    # 32      치즈 프랜치 프라이                                            [옵션 없음]
    # 33   치킨 매운맛(윙2+봉2)                                            [옵션 없음]
    # 34   치킨 순한맛(윙2+봉2)                                            [옵션 없음]
    # 35        코울슬로 샐러드                                            [옵션 없음]
    # 36       콘 치즈볼(3개)                                            [옵션 없음]
    # 37            콘샐러드                                            [옵션 없음]
    # 38              콜라                                            [옵션 없음]
    # 39    통가슴 후라이드(1개)                                            [옵션 없음]
    # 40         프랜치 프라이                                            [옵션 없음]
    # 41          프랭크 버거                      {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}
    # 42       프랭크 버거 세트  {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ...
    # 43          피넛 슈가볼                                            [옵션 없음]
    # 44   후라이드 아이스볼(1개)                    {'[필수 주문] 아이스볼 선택 ': '아이스볼 선택'}
    # 45   후라이드 아이스볼(2개)                    {'[필수 주문] 아이스볼 선택 ': '아이스볼 선택'}

    loader = DataFrameLoader(df_2, page_content_column="상품명")
    docs = loader.load()

    print('---------------- docs ----------------')
    print(docs)  # [Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='JG 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='JG 버거 세트'), Document(metadata={'옵션': "{'[필수 주문] 체다치즈 선택 ': '체다치즈 선택'}"}, page_content='JG 치즈 스틱(1개)'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='SG 불고기 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='SG 불고기 버거 세트'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='SG 크림 치즈 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='SG 크림 치즈 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='게다리살 튀김(3개)'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='더블 치즈 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='더블 치즈 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='더치즈볼(3개)'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='머쉬룸 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='머쉬룸 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='밀크쉐이크'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='베이컨 치즈 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='베이컨 치즈 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='사이다'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='생수 500ml'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='쉬림프 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='쉬림프 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='스파이시 텐더(2개)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='아메리카노(HOT)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='아메리카노(ICE)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='오딘에 반할라 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='오렌지'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='자이언트 통닭다리(1개)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='제로 콜라'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='초코칩 쿠키'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='치즈 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='치즈 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='치즈 스틱(3개)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='치즈 쿠키'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='치즈 프랜치 프라이'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='치킨 매운맛(윙2+봉2)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='치킨 순한맛(윙2+봉2)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='코울슬로 샐러드'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='콘 치즈볼(3개)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='콘샐러드'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='콜라'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='통가슴 후라이드(1개)'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='프랜치 프라이'), Document(metadata={'옵션': "{'[필수 주문] 사이즈 선택 ': '사이즈 선택'}"}, page_content='프랭크 버거'), Document(metadata={'옵션': "{'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}"}, page_content='프랭크 버거 세트'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='피넛 슈가볼'), Document(metadata={'옵션': "{'[필수 주문] 아이스볼 선택 ': '아이스볼 선택'}"}, page_content='후라이드 아이스볼(1개)'), Document(metadata={'옵션': "{'[필수 주문] 아이스볼 선택 ': '아이스볼 선택'}"}, page_content='후라이드 아이스볼(2개)')]
    print(len(docs))  # 46
    print(docs[0])

    from langchain_community.embeddings import HuggingFaceBgeEmbeddings
    from langchain_community.vectorstores import FAISS

    encode_kwargs={'normalize_embeddings':True}
    model_kwargs={'device':'cpu'}

    hf = HuggingFaceBgeEmbeddings(
        model_name=CFG.embedding_model,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )

    db = FAISS.from_documents(docs, hf)

    db.save_local(f"{CFG.output_path}/{store}_faiss")

    import pickle
    with open(f"{CFG.output_path}/{store}_docs.pkl", "wb") as f:
        pickle.dump(docs, f)

    # db = FAISS.load_local(f"/home/user14/beaver/log/db/{store}_faiss", hf, allow_dangerous_deserialization=True)

    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": CFG.retriever_k}
    )

    bm25_retriever = BM25Retriever.from_documents(docs)
    bm25_retriever.k = CFG.retriever_k

    ensemble_retriever = EnsembleRetriever(
        retrievers=[retriever, bm25_retriever],
        weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
    )

    data_df__ = data_df_[data_df_['store'] == store].copy()
    print('---------------- data_df__.shape ----------------')
    print(data_df__.shape)

    def format_docs(docs):
        print('---------------- docs ----------------')
        print(docs)  # ["베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}", "치즈 버거 세트: {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}", "더블 치즈 버거 세트: {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}", "프랭크 버거 세트: {'[필수 주문] 사이드 선택 ': '사이드 선택', '[필수 주문] 사이즈 선택 ': '사이즈 선택', '[필수 주문] 음료선택 ': '음료선택'}", "베이컨 치즈 버거: {'[필수 주문] 사이즈 선택 ': '사이즈 선택'}", '치즈 프랜치 프라이: [옵션 없음]', '프랜치 프라이: [옵션 없음]']
        return "\n".join(f"- {doc}".replace('"', '') for doc in docs[:CFG.retriever_k])

    def invoke_format(example):
        # example:
        # store                                                           프랭크버거
        # before_slot         {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...
        # current_slot        {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...
        # before_input                                     베이컨 치즈 버거 단품 하나 주세요.
        # before_response     네 ,베이컨 치즈 버거 단품 추가해드리겠습니다. 고객님! L/R 중에 사이즈를 선택...
        # current_input                                       그냥 단품 말고 세트로 주세요.
        # current_response    네 세트로 주문 도와드리겠습니다 고객님! 베이컨 치즈 버거 세트의 사이즈, 사이드,...

        before_slot = example['before_slot']

        text1 = " ".join([i.split('\'')[1] for i in before_slot.split("상품명': ")[1:]])
        # text1: 베이컨 치즈 버거
        text2 = "" if example['before_input'] == "이전 대화 없음" else example['before_input']
        # text2: 베이컨 치즈 버거 단품 하나 주세요.
        text3 = example['current_input']
        # text3: 그냥 단품 말고 세트로 주세요.
        text = text1 + " " + text2 + " " + text3
        # text: 베이컨 치즈 버거 베이컨 치즈 버거 단품 하나 주세요. 그냥 단품 말고 세트로 주세요.

        print('---------------- text ----------------')
        print('example:', example)
        print('text1:', text1)
        print('text2:', text2)
        print('text3:', text3)
        print('text:', text)

        return text

    print('---------------- format_docs, invoke_format ----------------')
    x = ensemble_retriever.invoke(invoke_format(data_df__.iloc[0]))
    print(x)
    print(format_docs([f"{i.page_content}: {i.metadata['옵션']}" for i in ensemble_retriever.invoke(invoke_format(data_df__.iloc[0]))]))

    data_df__['retriever'] = data_df__.apply(lambda x: format_docs([f"{i.page_content}: {i.metadata['옵션']}" for i in ensemble_retriever.invoke(invoke_format(x))]), axis=1)
    data_df__['all_menus'] = data_df__.apply(lambda x: menu_str, axis=1)
    print('---------------- data_df__ ----------------')
    print(data_df__)
    data_df___ = pd.concat([data_df___, data_df__], axis=0)


data_df___ = data_df___.reset_index(drop=True)

NameError: name 'pd' is not defined

In [5]:
data_df___.to_csv(f'/home/user14/beaver/data_test/dataset_v{CFG.version}.csv', index=False)

In [6]:
from datasets import Dataset 
dataset = Dataset.from_pandas(data_df___)


from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=25, shuffle=True, random_state=CFG.seed)

for i, (train_index, val_index) in enumerate(skf.split(dataset['current_input'], dataset['store'])):
    train_dataset = dataset.select(train_index)
    val_dataset = dataset.select(val_index)
    
train_dataset.to_json(f"/home/user06/beaver/data_test/dataset/beaver_v{CFG.version}_train.json")
val_dataset.to_json(f"/home/user06/beaver/data_test/dataset/beaver_v{CFG.version}_val.json")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 121.47ba/s]


522566

In [7]:
import datasets
test_dataset = datasets.load_dataset('json', data_files=f'/home/user06/beaver/data_test/dataset/beaver_v{CFG.version}_val.json')

Generating train split: 42 examples [00:00, 2649.55 examples/s]


In [8]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['store', 'before_slot', 'current_slot', 'before_input', 'before_response', 'current_input', 'current_response', 'retriever', 'all_menus'],
        num_rows: 42
    })
})

In [9]:
data_df__

,store,before_slot,current_slot,before_input,before_response,current_input,current_response,retriever,all_menus
434,피자먹다,"{'매장포장여부': None, '주문내역': [None], '결제수단': None}","{'매장포장여부': None, '주문내역': [None], '결제수단': None}",이전 대화 없음,이전 대화 없음,스위트 프라이드 치킨 피자에 추가할 수 있는 옵션은 뭐가 있나요?,"네 고객님, 스위트 프라이드 치킨 피자에 추가할 수 있는 옵션은 치즈 추가가 있습니다.",- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
435,피자먹다,"{'매장포장여부': None, '주문내역': [None], '결제수단': None}","{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...",스위트 프라이드 치킨 피자에 추가할 수 있는 옵션은 뭐가 있나요?,"네 고객님, 스위트 프라이드 치킨 피자에 추가할 수 있는 옵션은 치즈 추가가 있습니다.",그러면 스위트 프라이드 치킨 피자에 치즈 추가해서 하나 주시겠어요? 고구마 콘치즈도...,스위트 프라이드 치킨 피자 하나에 치즈 추가 옵션 고구마 콘치즈 옵션 추가해드렸습니...,- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
436,피자먹다,"{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...",그러면 스위트 프라이드 치킨 피자에 치즈 추가해서 하나 주시겠어요? 고구마 콘치즈도...,스위트 프라이드 치킨 피자 하나에 치즈 추가 옵션 고구마 콘치즈 옵션 추가해드렸습니...,음료는 뭐가 있나요?,"저희 매장에서 판매하는 음료는 펩시, 펩시 제로, 칠성 사이다, 칠성 사이다 제로,...",- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
437,피자먹다,"{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...",음료는 뭐가 있나요?,"저희 매장에서 판매하는 음료는 펩시, 펩시 제로, 칠성 사이다, 칠성 사이다 제로,...",탐스 제로 오렌지 맛 두 개도 추가해주세요.,탐스 제로(오렌지) 두 개 추가하셨습니다. 더 추가하실 상품이 없으면 매장 포장 여...,- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
438,피자먹다,"{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...",탐스 제로 오렌지 맛 두 개도 추가해주세요.,탐스 제로(오렌지) 두 개 추가하셨습니다. 더 추가하실 상품이 없으면 매장 포장 여...,아 갈릭디핑소스 3개 추가해줘요.,네 갈릭 디핑 소스 3개 추가했습니다. 더. 추가하실 상품이 없으면 매장 포장 여부...,- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
439,피자먹다,"{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...",아 갈릭디핑소스 3개 추가해줘요.,네 갈릭 디핑 소스 3개 추가했습니다. 더. 추가하실 상품이 없으면 매장 포장 여부...,이정도면 될 거 같아요.,"네 고객님, 주문 완료 도와드리겠습니다!마지막으로 매장 포장 여부와 결제 수단을 선...",- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
440,피자먹다,"{'매장포장여부': None, '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...",이정도면 될 거 같아요.,"네 고객님, 주문 완료 도와드리겠습니다!마지막으로 매장 포장 여부와 결제 수단을 선...",여기서 먹고 갈래요.,매장에서 음식 준비해드리겠습니다. 결제 수단 말씀해주시겠어요?,- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
441,피자먹다,"{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...",여기서 먹고 갈래요.,매장에서 음식 준비해드리겠습니다. 결제 수단 말씀해주시겠어요?,"잠시만요, 아란치니 4조각도 하나 추가해주실래요?","네 그럼요 고객님, 아란치니 4조각 하나 추가해드리겠습니다. 결제 수단 말씀해주시겠어요?",- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
442,피자먹다,"{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...","잠시만요, 아란치니 4조각도 하나 추가해주실래요?","네 그럼요 고객님, 아란치니 4조각 하나 추가해드리겠습니다. 결제 수단 말씀해주시겠어요?",감사합니다. 카카오페이로 결제할게요.,"네 고객님, 주문 확인후에 결제 도와드리겠습니다! 이렇게 주문 도와드릴까요?<|sh...",- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
443,피자먹다,"{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...","{'매장포장여부': '매장', '주문내역': [{'상품명': '스위트 프라이드 치킨...",감사합니다. 카카오페이로 결제할게요.,"네 고객님, 주문 확인후에 결제 도와드리겠습니다! 이렇게 주문 도와드릴까요?<|sh...",감사합니다. 카카오페이로 결제할게요.,"스위트 프라이드 치킨 피자에 치즈 추가하신 거 하나, 고구마 콘치즈 하나, 탐스 제...",- 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']...,"- 사이드: 감자 튀김, 고구마 콘치즈, 마약 콘치즈, 바삭 통새우(생새우깡), 베..."
